In [4]:
# -*- coding: utf-8 -*-
import os
import sys
from readmdict import MDX
from glob import glob
from bs4 import BeautifulSoup
import re

# 加载.mdx文件,修改此处，可以更换不同的mdx；
# 注意： 路径下必须同时包含mdx和css文件。
dict_dir = 'Vocabulary/新东方英语词根词缀'
#dict_dir = 'Vocabulary/etymology'
mdx_name = glob(os.path.join(dict_dir,'*.mdx'))[0]
css_name = glob(os.path.join(dict_dir,'*.css'))[0]
print('mdx_name = ', mdx_name)
print('css_name = ', css_name)


mdx_name =  Vocabulary/新东方英语词根词缀/NewOrientalRootsAffixes.mdx
css_name =  Vocabulary/新东方英语词根词缀/NewOrientalRootsAffixes.css


In [5]:


idx_to_prefix = {}
prefix_to_idx = {}

def update_idx_prefix_map(class_id,prefix):
    global idx_to_prefix, prefix_to_idx
    idx_to_prefix[class_id] = []
    for p_ in prefix.split(','):
        p = p_.strip()
        idx_to_prefix[class_id].append(p)
        if p not in prefix_to_idx.keys():
            prefix_to_idx[p] = []
        prefix_to_idx[p].append(class_id)
    
def detect_first_chinese_position(text):
    chinese_pattern = re.compile(r'[\u4e00-\u9fa5]')
    match = chinese_pattern.search(text)
    if match:
        return match.start()
    else:
        return -1

def split_english_chinese(text):
    c_id = detect_first_chinese_position(text)
    if c_id < 0:
        return text, None
    english = text[:c_id]
    chinese = text[c_id:]
    return english, chinese
    
def get_all_roots(soup,v=False):
    '''
    root_dict = {'class_id':{'root':meaning}, ...}
    {
        2: {'ab-, abs- ': '表示“去掉，离去；从”'}, 
        3: {'ab-, ac-, af-, ag-, ap-, ar-, as-, at- ': '等加在同辅音词根前，表加强'}
    }
    ############################################################################
    idx_to_prefix = {}
        {
            2: ['ab-', 'abs-'], 
            3: ['ab-', 'ac-', 'af-', 'ag-', 'ap-', 'ar-', 'as-', 'at-']
        }
    ############################################################################
    prefix_to_idx = {}
        {
            'ab-': [2, 3], 
            'abs-': [2], 
            'ac-': [3], 
            'af-': [3], 
            'ag-': [3], 
            'ap-': [3], 
        }   
    '''
    roots_list  = soup.find_all('h2')
    root_dict = {}
    for h2 in roots_list:
        class_name = h2.contents[0]['class'][0]
        class_id = int(h2.contents[0].string.split('.')[0])
        prefix, word_meaning = split_english_chinese(h2.contents[1].string)  
        root_dict[class_id] = {}
        root_dict[class_id][str(prefix)] = str(word_meaning)
        update_idx_prefix_map(class_id, prefix)
    if v:
        print(root_dict)
    

def get_all_head_words(soup,v=False):
    '''
    head_word_dict = {'id':word,...}
    {
        'NewOrientalHeadWord_1_abnormal': 'abnormal',
        'NewOrientalHeadWord_1_abrupt': 'abrupt', 
    }
    '''
    head_words = soup.find_all('div', class_='head-words') 
    head_word_dict = {}
    for p in head_words:
        for a in p.find_all('a'):
            k = a['href'].replace('#','')
            v = a.string
            head_word_dict[k] = v
    print(head_word_dict)
    

def get_word_info(soup, v=False):
    '''
    word_dict = {'id_1':{}, 'id_2':{}, ...}
    {------
     id: NewOrientalHeadWord_1_abnormal 
        {======
         word :		 abnormal
         yin_biao :		 ［æbˈnɔ:ml］
         ci_xing :		 a.
         meaning :		 反常的
         derive :		 ab＋normal（正常的）→反常的
        }======
     }-----
    '''
    words_entries = soup.find_all('div', class_='word-entry') 
    word_dict = {}
    for div in words_entries:
        word_dict[div['id']] = {}
        d = word_dict[div['id']]
        for p in div.find_all('p'):
            if len(p.attrs)== 0:
                word = p.find('span',class_='bold').text.strip()
                yin_biao = p.a.find('span',class_='cn-kindle-yinbiao').text.strip()
                d['word'] = word
                d['yin_biao'] = yin_biao
            else:
                if p['class'][0] == 'meaning':
                    ci_xing = p.find('span', class_='textItalic').string
                    meaning = p.contents[-1].strip()
                    d['ci_xing'] = ci_xing
                    d['meaning'] = meaning
                if p['class'][0] == 'derive':
                    derive = p.string.strip()
                    d['derive'] = derive
        if v:
            for k, v in word_dict.items():
                print(k, '*'*50)
                for k_, v_ in v.items():
                    print(k_,':\t\t',v_)




In [6]:
mdx = MDX(mdx_name)
print(mdx.__dir__())
'''
for k in mdx.__dict__['_key_list']:
    print(k[0], k[1].decode())
'''
headwords = [*mdx]
items = [*mdx.items()]


for k, v in mdx.items():
    print(k.decode('utf-8'))
    soup = BeautifulSoup(v.decode('utf-8'), 'html5lib')
    get_word_info(soup,True)


['_fname', '_encoding', '_passcode', '_key_block_offset', '_encrypt', '_stylesheet', '_version', '_number_width', '_number_format', 'header', '_num_entries', '_record_block_offset', '_key_list', '_substyle', '__module__', '__doc__', '__init__', 'items', '_substitute_stylesheet', '_decode_record_block', '__len__', '__iter__', 'keys', '_read_number', '_parse_header', '_decode_key_block_info', '_decode_key_block', '_split_key_block', '_read_header', '_read_keys', '_read_keys_brutal', '__dict__', '__weakref__', '__repr__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__new__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']
a-
NewOrientalHeadWord_0_amoral **************************************************
word :		 amoral
yin_biao :		 ［ˌeɪˈmɒrəl］
ci_xing :		 a.
meaning :		 非道德性的
derive :		 a＋moral（道德的）→非道德性的；注意：immoral意为“不道德的”
N

NewOrientalHeadWord_0_amoral **************************************************
word :		 amoral
yin_biao :		 ［ˌeɪˈmɒrəl］
ci_xing :		 a.
meaning :		 非道德性的
derive :		 a＋moral（道德的）→非道德性的；注意：immoral意为“不道德的”
NewOrientalHeadWord_0_asocial **************************************************
word :		 asocial
yin_biao :		 ［ˌeɪˈsəʊʃl］
ci_xing :		 a.
meaning :		 不合群的
derive :		 a＋social（好社交的）→不好社交的→不合群的
NewOrientalHeadWord_0_atypical **************************************************
word :		 atypical
yin_biao :		 ［ˌeɪˈtɪpɪkl］
ci_xing :		 a.
meaning :		 非典型的
derive :		 a＋typical（典型的）→非典型的
NewOrientalHeadWord_0_acentric **************************************************
word :		 acentric
yin_biao :		 ［əˈsentrɪk］
ci_xing :		 a.
meaning :		 无中心的
derive :		 a＋centric（中心的）→无中心的
NewOrientalHeadWord_0_apathy **************************************************
word :		 apathy
yin_biao :		 ［ˈæpəθi］
ci_xing :		 n.
meaning :		 冷漠
derive :		 a＋pathy（感觉）→没感觉→冷漠
NewOrientalHeadWord_0_aperiodic ******************

word :		 asymmetry
yin_biao :		 ［ˌeɪˈsɪmətri］
ci_xing :		 n.
meaning :		 不对称
derive :		 a＋symmetry（对称）→不对称
NewOrientalHeadWord_0_afloat **************************************************
word :		 afloat
yin_biao :		 ［əˈfləʊt］
ci_xing :		 a.
meaning :		 漂浮的
derive :		 a＋float（漂浮）→漂浮的
NewOrientalHeadWord_0_ahead **************************************************
word :		 ahead
yin_biao :		 ［əˈhed］
ci_xing :		 ad.
meaning :		 在前面
derive :		 a＋head（前端，前面）→在前面
NewOrientalHeadWord_0_alive **************************************************
word :		 alive
yin_biao :		 ［əˈlaɪv］
ci_xing :		 a.
meaning :		 活的
derive :		 a＋live（活）→活的
NewOrientalHeadWord_0_aside **************************************************
word :		 aside
yin_biao :		 ［əˈsaɪd］
ci_xing :		 ad.
meaning :		 在旁边
derive :		 a＋side（旁边）→在旁边
NewOrientalHeadWord_0_asleep **************************************************
word :		 asleep
yin_biao :		 ［əˈsli:p］
ci_xing :		 a.
meaning :		 睡着的
derive :		 a＋sleep（睡觉）→睡着的
NewOrientalHeadWord

NewOrientalHeadWord_0_asleep **************************************************
word :		 asleep
yin_biao :		 ［əˈsli:p］
ci_xing :		 a.
meaning :		 睡着的
derive :		 a＋sleep（睡觉）→睡着的
NewOrientalHeadWord_0_away **************************************************
word :		 away
yin_biao :		 ［əˈweɪ］
ci_xing :		 ad.
meaning :		 远离，离
derive :		 a＋way（路）→在路上→远离
NewOrientalHeadWord_0_akin **************************************************
word :		 akin
yin_biao :		 ［əˈkɪn］
ci_xing :		 a.
meaning :		 同族的
derive :		 a＋kin（亲属，同族）→同族的
NewOrientalHeadWord_0_alight **************************************************
word :		 alight
yin_biao :		 ［əˈlaɪt］
ci_xing :		 a.
meaning :		 点燃的
derive :		 a＋light（点燃）→点燃的
NewOrientalHeadWord_0_ashore **************************************************
word :		 ashore
yin_biao :		 ［əˈʃɔ:(r)］
ci_xing :		 ad.
meaning :		 在岸上
derive :		 a＋shore（岸）→在岸上
NewOrientalHeadWord_0_alike **************************************************
word :		 alike
yin_biao :		 ［əˈlaɪk］
ci_xing 

NewOrientalHeadWord_0_amoral **************************************************
word :		 amoral
yin_biao :		 ［ˌeɪˈmɒrəl］
ci_xing :		 a.
meaning :		 非道德性的
derive :		 a＋moral（道德的）→非道德性的；注意：immoral意为“不道德的”
NewOrientalHeadWord_0_asocial **************************************************
word :		 asocial
yin_biao :		 ［ˌeɪˈsəʊʃl］
ci_xing :		 a.
meaning :		 不合群的
derive :		 a＋social（好社交的）→不好社交的→不合群的
NewOrientalHeadWord_0_atypical **************************************************
word :		 atypical
yin_biao :		 ［ˌeɪˈtɪpɪkl］
ci_xing :		 a.
meaning :		 非典型的
derive :		 a＋typical（典型的）→非典型的
NewOrientalHeadWord_0_acentric **************************************************
word :		 acentric
yin_biao :		 ［əˈsentrɪk］
ci_xing :		 a.
meaning :		 无中心的
derive :		 a＋centric（中心的）→无中心的
NewOrientalHeadWord_0_apathy **************************************************
word :		 apathy
yin_biao :		 ［ˈæpəθi］
ci_xing :		 n.
meaning :		 冷漠
derive :		 a＋pathy（感觉）→没感觉→冷漠
NewOrientalHeadWord_0_aperiodic ******************

...

derive :		 ac＋cumul（堆积）＋ate→不断堆积→积累
NewOrientalHeadWord_2_affirm **************************************************
word :		 affirm
yin_biao :		 ［əˈfɜ:m］
ci_xing :		 v.
meaning :		 肯定
derive :		 af＋firm（坚定）→非常坚定→肯定
NewOrientalHeadWord_2_aggravate **************************************************
word :		 aggravate
yin_biao :		 ［ˈæɡrəveɪt］
ci_xing :		 v.
meaning :		 恶化
derive :		 ag＋grav（重）＋ate→（病等）加重→恶化
NewOrientalHeadWord_2_appease **************************************************
word :		 appease
yin_biao :		 ［əˈpi:z］
ci_xing :		 v.
meaning :		 抚慰
derive :		 ap＋peas（平和）＋e→使（人）心态平和→抚慰
NewOrientalHeadWord_2_applause **************************************************
word :		 applause
yin_biao :		 ［əˈplɔ:z］
ci_xing :		 n.
meaning :		 鼓掌
derive :		 ap＋plaus（鼓掌）＋e→鼓掌
NewOrientalHeadWord_2_appoint **************************************************
word :		 appoint
yin_biao :		 ［əˈpɔɪnt］
ci_xing :		 v.
meaning :		 指定
derive :		 ap＋point（指）→指定
NewOrientalHeadWord_2_appreciate *************

ci_xing :		 v.
meaning :		 扩大
derive :		 ag＋grand（大）＋ise（使）→一再使变大→扩大
NewOrientalHeadWord_2_aggression **************************************************
word :		 aggression
yin_biao :		 ［əˈɡreʃn］
ci_xing :		 n.
meaning :		 侵略，进攻；攻击性
derive :		 ag＋gress（行走）＋ion→不停地往前走→进攻
NewOrientalHeadWord_2_assiduous **************************************************
word :		 assiduous
yin_biao :		 ［əˈsɪdjuəs］
ci_xing :		 a.
meaning :		 勤奋的
derive :		 as＋sid（坐）＋u＋ous→一再坐着（学习）→勤奋的
NewOrientalHeadWord_2_attest **************************************************
word :		 attest
yin_biao :		 ［əˈtest］
ci_xing :		 v.
meaning :		 证实
derive :		 at＋test（试验）→一再试验→证实
NewOrientalHeadWord_2_accelerate **************************************************
word :		 accelerate
yin_biao :		 ［əkˈseləreɪt］
ci_xing :		 v.
meaning :		 加速
derive :		 ac＋celer（速度）＋ate→加快速度→加速
NewOrientalHeadWord_2_accompany **************************************************
word :		 accompany
yin_biao :		 ［əˈkʌmpəni］
ci_xing :		 v.
meaning :		 陪

ci_xing :		 v.
meaning :		 积累
derive :		 ac＋cumul（堆积）＋ate→不断堆积→积累
NewOrientalHeadWord_2_affirm **************************************************
word :		 affirm
yin_biao :		 ［əˈfɜ:m］
ci_xing :		 v.
meaning :		 肯定
derive :		 af＋firm（坚定）→非常坚定→肯定
NewOrientalHeadWord_2_aggravate **************************************************
word :		 aggravate
yin_biao :		 ［ˈæɡrəveɪt］
ci_xing :		 v.
meaning :		 恶化
derive :		 ag＋grav（重）＋ate→（病等）加重→恶化
NewOrientalHeadWord_2_appease **************************************************
word :		 appease
yin_biao :		 ［əˈpi:z］
ci_xing :		 v.
meaning :		 抚慰
derive :		 ap＋peas（平和）＋e→使（人）心态平和→抚慰
NewOrientalHeadWord_2_applause **************************************************
word :		 applause
yin_biao :		 ［əˈplɔ:z］
ci_xing :		 n.
meaning :		 鼓掌
derive :		 ap＋plaus（鼓掌）＋e→鼓掌
NewOrientalHeadWord_2_appoint **************************************************
word :		 appoint
yin_biao :		 ［əˈpɔɪnt］
ci_xing :		 v.
meaning :		 指定
derive :		 ap＋point（指）→指定
NewOrientalHeadW

yin_biao :		 ［əˈflɪkʃn］
ci_xing :		 n.
meaning :		 折磨；苦难
derive :		 af＋flict（打）＋ion→一再打→折磨
NewOrientalHeadWord_2_aggrandise **************************************************
word :		 aggrandise
yin_biao :		 ［əˈɡrændaɪz］
ci_xing :		 v.
meaning :		 扩大
derive :		 ag＋grand（大）＋ise（使）→一再使变大→扩大
NewOrientalHeadWord_2_aggression **************************************************
word :		 aggression
yin_biao :		 ［əˈɡreʃn］
ci_xing :		 n.
meaning :		 侵略，进攻；攻击性
derive :		 ag＋gress（行走）＋ion→不停地往前走→进攻
NewOrientalHeadWord_2_assiduous **************************************************
word :		 assiduous
yin_biao :		 ［əˈsɪdjuəs］
ci_xing :		 a.
meaning :		 勤奋的
derive :		 as＋sid（坐）＋u＋ous→一再坐着（学习）→勤奋的
NewOrientalHeadWord_2_attest **************************************************
word :		 attest
yin_biao :		 ［əˈtest］
ci_xing :		 v.
meaning :		 证实
derive :		 at＋test（试验）→一再试验→证实
NewOrientalHeadWord_135_differentiate **************************************************
word :		 differentiate
yin_biao :		 ［ˌdɪfə

AttributeError: 'NoneType' object has no attribute 'strip'

In [ ]:
mdx = MDX(mdx_name)
headwords = [*mdx]       # 单词名列表
items = [*mdx.items()]   # 释义html源码列表
data = [*mdx.items()][3][1].decode('utf-8')

#decoded_data = {key.decode('utf-8'): value.decode('utf-8') for key, value in data.items()}
#print(data)



text = data

soup = BeautifulSoup(text, 'html5lib')
get_all_roots(soup,v=False)
get_all_head_words(soup,v=False)
get_word_info(soup)


print(idx_to_prefix)
print(prefix_to_idx)

In [ ]:
'''
for idx in range(10):
    data = [*mdx.items()][idx][1].decode('utf-8')
    f_name = str(idx)+'.html'
    with open(f_name, 'w')as f:
        f.write(data)
'''
